In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly
## import chart_studio.plotly as py
import os, glob 

pd.options.display.max_columns = None
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import jupyter_dash as jd

#### Read in the attribute files into a pandas df

In [ ]:
### Read in sequence 050 as  the start of our df: 
file = './qc_headers/p001j00_rf_hyd_hdr_1072050.att'
## 23 coulmns
cols = ('SHOT', 'FFID', 'GUNFLAG', 'TYPE', 'OFFSET', 'SPCDELTA', 'SHT_X', 'SHT_Y', 'REC_X', 'REC-Y', 'RELEV', 'SELEV', 'SDEPTH', 'RDATUM', 'SDATUM', 'SWEVEL', 'TIDE', 'MSL_TIDE', 'WDS', 'WDR', 'W_DEPTH', 'WBT', 'SLINE')
## Define the column widths:
colspec = [(8, 12), (20, 24), (34, 36), (46, 48), (57, 60), (61, 72), (73, 86), (87, 101), (102, 115), (116, 130), (132, 139), (145, 151), (157, 163), (176, 178), (188, 190), (198, 203), (205, 211), (217, 223), (227, 238), (239, 250), (256, 263), (270, 275), (279, 287)]
### Read in the header file - infer doesn't work too well....: 
df = pd.read_fwf(file, colspecs=colspec, skiprows=1, names=cols )
df.head()

In [ ]:
df.describe()

In [4]:
files = glob.glob(os.path.join("./qc_headers/p*.att"))
print(files)

['./qc_headers\\p001j00_rf_hyd_hdr_1072050.att', './qc_headers\\p001j00_rf_hyd_hdr_1088046.att', './qc_headers\\p001j00_rf_hyd_hdr_1104009.att', './qc_headers\\p001j00_rf_hyd_hdr_1120054.att', './qc_headers\\p001j00_rf_hyd_hdr_1136052.att', './qc_headers\\p001j00_rf_hyd_hdr_1152044.att', './qc_headers\\p001j00_rf_hyd_hdr_1168042.att', './qc_headers\\p001j00_rf_hyd_hdr_1184040.att', './qc_headers\\p001j00_rf_hyd_hdr_1200038.att', './qc_headers\\p001j00_rf_hyd_hdr_1200067.att', './qc_headers\\p001j00_rf_hyd_hdr_1216036.att', './qc_headers\\p001j00_rf_hyd_hdr_1232034.att', './qc_headers\\p001j00_rf_hyd_hdr_1232069.att', './qc_headers\\p001j00_rf_hyd_hdr_1248032.att', './qc_headers\\p001j00_rf_hyd_hdr_1264030.att', './qc_headers\\p001j00_rf_hyd_hdr_1280028.att', './qc_headers\\p001j00_rf_hyd_hdr_1280056.att', './qc_headers\\p001j00_rf_hyd_hdr_1296021.att', './qc_headers\\p001j00_rf_hyd_hdr_1296066.att', './qc_headers\\p001j00_rf_hyd_hdr_1312026.att', './qc_headers\\p001j00_rf_hyd_hdr_13280

In [ ]:
## For loop to read in the rest of the att files:    
for i in range(1,len(files)):
    data = pd.read_fwf(files[i], colspecs=colspec, skiprows=1, names=cols )
    df_temp = pd.DataFrame(data)
    df = pd.concat([df,df_temp], axis=0)
print(df)

In [ ]:
print(df.head())
print(df.tail())


In [7]:
from tabulate import tabulate

def to_fwf(df, fname):
    content = tabulate(df.values.tolist(), list(df.columns), tablefmt="plain")
    open(fname, "w").write(content)

pd.DataFrame.to_fwf = to_fwf

In [8]:
df.to_fwf("4693tot4_attributes.att")

In [ ]:

df['ACQSEQ'] = df['SLINE'].astype(str).apply(lambda x:x[-3:]).astype(int)
df.head(20)

In [ ]:
df = df.sort_values(by=['ACQSEQ', 'FFID'])
df.head()

In [ ]:
print(df['FFID'])

In [11]:
df1 = df.dropna()

In [ ]:
df.loc[df['ACQSEQ'] == 2]

In [13]:
dfc = df1.groupby('ACQSEQ')['FFID']
df1 = df1.assign(FFID_MIN=dfc.transform(min), FFID_MAX=dfc.transform(max))

In [14]:
dfc = df.groupby('ACQSEQ')['FFID']
df = df.assign(FFID_MIN=dfc.transform(min), FFID_MAX=dfc.transform(max))

In [15]:
dfc = df1.groupby('ACQSEQ')['SHOT']
df1 = df1.assign(SP_MIN=dfc.transform(min), SP_MAX=dfc.transform(max))
dfc = df.groupby('ACQSEQ')['SHOT']
df = df.assign(SP_MIN=dfc.transform(min), SP_MAX=dfc.transform(max))

In [16]:
gk = df.groupby('ACQSEQ')
min_dict = {gk['ACQSEQ']:gk['FFID'].min('FFID')}
max_dict = {gk['ACQSEQ']:gk['FFID'].max('FFID')}

In [ ]:
display(df.tail(10))
display(df1.head(10))

In [18]:
df.to_fwf("4693tot4_attributes_extra_headers.att")

In [19]:
df1.to_fwf("4693tot4_attributes_removed_NAN.att")

#### Graphical Representation of the Attributes

In [ ]:
### Overlapping shots: 
fig = px.line(df, x="FFID", y="SPCDELTA", title='SPCDELTA', 
              animation_frame="ACQSEQ", 
              range_x=[df['FFID_MIN'],df['FFID_MAX']], range_y=[6000,11000] )
fig.add_hrect(y0=6000, y1=7000, line_width=0, fillcolor="red", opacity=0.2)
fig.write_html("2022057_SPCDELTA_QC.html")
fig.show()

##### Putting the SPCDelta Data into Dashboard ..... and designing the layout

In [28]:
proj_title = '2022057 / 4693tot4 Gengibre'
acq_min = df1['ACQSEQ'].min()
acq_max = df['ACQSEQ'].max()

In [ ]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.PULSE])

app.layout = html.Div([
    ## Header
    dbc.Col([
        html.H2(f'SPARK Header QC for {proj_title}', className="bg-primary text-white p-2 mb-2 text-center"),
        # html.H3(f'For Sequences: {acq_min} - {acq_max}'),
    ], style={'textAlign': 'center'}),

    #html.Br(),

    dbc.Row([
        dbc.Col([
            dcc.Dropdown(id='ACQSEQ_dropdown', options=[{'label': ACQSEQ, 'value': str(ACQSEQ)}
                                     for ACQSEQ in df1['ACQSEQ'].drop_duplicates().sort_values()])
        ], width=2),
        dbc.Col([
                dcc.Graph(id='SPCDELTA_line'),
                #html.Br(),
                dcc.Graph(id='WBT_GUN'),
                #html.Br(),
                dcc.Graph(id='OFFSET')
                ], width=5),
        dbc.Col([
                dcc.Graph(id='W_DEPTH'),
                #html.Br(),
                dcc.Graph(id='TIDE'),
                #html.Br(),
                dcc.Graph(id='TYPE_bar')
                ], width=5)

    ])
])     

## Callback for SPCDELTA line plot:
@app.callback(Output('SPCDELTA_line', 'figure'), Input('ACQSEQ_dropdown', 'value'))
def display_spc_delta(ACQSEQ):
    if not ACQSEQ:
       raise PreventUpdate
    # data = df1[[int(ACQSEQ), 'FFID', 'SPCDELTA']]
    data = df1[df1['ACQSEQ'] == int(ACQSEQ)]
    # Line plot for SPCDELTA
    chart = px.line(data,
                   x='FFID',
                   y='SPCDELTA', 
                   range_x=['FFID_MIN','FFID_MAX'], range_y=[6000,11000], 
                   title=f'SPC_DELTA for Sequence: {ACQSEQ}'
                   )
    chart.add_hrect(y0=6000, y1=7000, line_width=0, fillcolor="red", opacity=0.2)
    return chart

## Callback for WBT with GUN line plot:
@app.callback(Output('WBT_GUN', 'figure'), Input('ACQSEQ_dropdown', 'value'))
def display_wbt_gun(ACQSEQ):
    if not ACQSEQ:
       raise PreventUpdate
    data = df1[df1['ACQSEQ'] == int(ACQSEQ)]
    ymin = data['WBT'].min() - 50
    ymax = data['WBT'].max() + 50
    chart = px.scatter(data,
                   x='FFID',
                   y='WBT',
                   color="GUNFLAG", 
                   range_x=['FFID_MIN','FFID_MAX'], 
                   range_y=[ymin,ymax], 
                   title=f'WBT for Each Gun - Sequence: {ACQSEQ}'
                   )
    #chart.add_hrect(y0=6000, y1=7000, line_width=0, fillcolor="red", opacity=0.2)
    return chart

## Callback for OFFSET line plot with GUNFLAG
@app.callback(Output('OFFSET', 'figure'), Input('ACQSEQ_dropdown', 'value'))
def display_wbt_gun(ACQSEQ):
    if not ACQSEQ:
       raise PreventUpdate
    data = df1[df1['ACQSEQ'] == int(ACQSEQ)]
    ymin = data['OFFSET'].min() - 10
    ymax = data['OFFSET'].max() + 10
    chart = px.line(data,
                   x='FFID',
                   y='OFFSET',
                   color="GUNFLAG", 
                   range_x=['FFID_MIN','FFID_MAX'], range_y=[ymin,ymax], 
                   title=f'OFFSET for each GUNFLAG - Sequence: {ACQSEQ}'
                   )
    #chart.add_hrect(y0=6000, y1=7000, line_width=0, fillcolor="red", opacity=0.2)
    return chart

## Callback for TIDE Scatter plot
@app.callback(Output('TIDE', 'figure'), Input('ACQSEQ_dropdown', 'value'))
def display_wbt_gun(ACQSEQ):
    if not ACQSEQ:
       raise PreventUpdate
    data = df1[df1['ACQSEQ'] == int(ACQSEQ)]
    ymin = data['MSL_TIDE'].min() - 0.1
    ymax = data['MSL_TIDE'].max() + 0.1
    chart = px.scatter(data,
                   x='FFID',
                   y='MSL_TIDE',
                   color="SWEVEL", 
                   range_x=['FFID_MIN','FFID_MAX'], range_y=[ymin,ymax], 
                   title=f'MSL TIDE with SWEVEL - Sequence: {ACQSEQ}'
                   )
    #chart.add_hrect(y0=6000, y1=7000, line_width=0, fillcolor="red", opacity=0.2)
    return chart

## Callback for the W_DEPTH line plot
@app.callback(Output('W_DEPTH', 'figure'), Input('ACQSEQ_dropdown', 'value'))
def display_wbt_gun(ACQSEQ):
    if not ACQSEQ:
       raise PreventUpdate
    data = df1[df1['ACQSEQ'] == int(ACQSEQ)]
    ymin = data['W_DEPTH'].min() - 50
    ymax = data['W_DEPTH'].max() + 50
    chart = px.line(data,
                   x='FFID',
                   y=['WDS', 'WDR', 'W_DEPTH'],
                   #color="GUNFLAG", 
                   range_x=['FFID_MIN','FFID_MAX'], range_y=[ymin,ymax], 
                   title=f'W_DEPTH / WDR / WDS - Sequence: {ACQSEQ}'
                   )
    #chart.add_hrect(y0=6000, y1=7000, line_width=0, fillcolor="red", opacity=0.2)
    return chart

## Callback for the TYPE bar chart
@app.callback(Output('TYPE_bar', 'figure'), Input('ACQSEQ_dropdown', 'value'))
def display_wbt_gun(ACQSEQ):
    if not ACQSEQ:
       raise PreventUpdate
    data = df1[df1['ACQSEQ'] == int(ACQSEQ)]
    ymin = 0
    ymax = 2
    chart = px.bar(data,
                   x='FFID',
                   y='TYPE',
                   color="GUNFLAG", 
                   range_x=['FFID_MIN','FFID_MAX'], range_y=[ymin,ymax],
                   color_continuous_scale=[(0, "red"), (1, "blue")], 
                   title=f'SHOT TYPE with GUNFLAG - Sequence: {ACQSEQ}'
                   )
    #chart.add_hrect(y0=6000, y1=7000, line_width=0, fillcolor="red", opacity=0.2)
    return chart


if __name__=='__main__':
    app.run_server(mode='inline', port = 8212)
   

In [79]:
### WBT 3D Function: 
def display_wbt3d():
    data1 = df1
    minx = data1['SHT_X'].min()
    miny = data1['SHT_Y'].min()
    maxx = data1['SHT_X'].max()
    maxy = data1['SHT_Y'].max()
    minz = data1['WBT'].min()
    maxz = data1['WBT'].max()
    z=pd.Series(data1['WBT'])
    x=pd.Series(data1['SHT_X'])
    y=pd.Series(data1['SHT_Y'])
    # 
    # print(x)
    chart1 = go.Figure(data = [go.Mesh3d(
                   z=z,
                   x=x,
                   y=y, 
                   opacity=0.25,
                   color='purple'
                   )])
    chart1.update_layout(title='WBT: Shot X-Y', autosize=False,
                  width=1000, height=1000,
                  scene = dict(
                        xaxis = dict(nticks=8, range=[minx,maxx],),
                        yaxis = dict(nticks=8, range=[miny,maxy],),
                        zaxis = dict(nticks=8, range=[maxz,minz],),
                        camera_eye = {"x": 1.5, "y": -1.5, "z": 0.5},
                        ),
                  margin=dict(l=35, r=20, b=35, t=40))
    return chart1

app.layout = html.Div([
        dcc.Graph(
            figure=display_wbt3d()
            #id='WBT_mesh',
            #className="figure"
        )
])

## 3D WBT with Gun Flag Display: 
def display_wbtgun3d():
    data1 = df1
    minx = data1['SHT_X'].min()
    miny = data1['SHT_Y'].min()
    maxx = data1['SHT_X'].max()
    maxy = data1['SHT_Y'].max()
    minz = data1['WBT'].min()
    maxz = data1['WBT'].max()
    z=pd.Series(data1['WBT'])
    x=pd.Series(data1['SHT_X'])
    y=pd.Series(data1['SHT_Y'])
    color = data1['GUNFLAG']
    # 
    # print(x)
    chart1 = px.line_3d(
                   z=z,
                   x=x,
                   y=y, 
                   color=color
                   )
    chart1.update_layout(title='WBT with GUNFLAG: Shot X-Y (Click legend to swap guns)', autosize=False,
                  width=1000, height=1000,
                  scene = dict(
                        xaxis = dict(nticks=8, range=[minx,maxx],),
                        yaxis = dict(nticks=8, range=[miny,maxy],),
                        zaxis = dict(nticks=8, range=[maxz,minz],),
                        camera_eye = {"x": 1.5, "y": -1.5, "z": 0.5},
                        ),
                  margin=dict(l=35, r=20, b=35, t=40))
    return chart1

In [ ]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.PULSE])

app.layout = html.Div([
    ## Header
    dbc.Col([
        html.H2(f'SPARK Header QC for {proj_title}', className="bg-primary text-white p-2 mb-2 text-center"),
        # html.H3(f'For Sequences: {acq_min} - {acq_max}'),
    ], style={'textAlign': 'center'}),

    #html.Br(),
    dbc.Tabs([
        dbc.Tab([
            dbc.Row([
                dbc.Col([
                    html.Br(),
                    html.H5(f'Pick A Sequence, Sucker:', style={'color': 'white', 'margin-left':'7px'}),
                    html.Br(),
                    dcc.Dropdown(id='ACQSEQ_dropdown', options=[{'label': ACQSEQ, 'value': str(ACQSEQ)}
                                            for ACQSEQ in df1['ACQSEQ'].drop_duplicates().sort_values()]),
                    html.Br(),
                    html.Br(),
                    html.H4(id='sline', style={'color': 'white', 'margin-left':'7px'} ),
                    html.H4(id='ffidrange', style={'color': 'white', 'margin-left':'7px'} ),
                    html.H4(id='sprange', style={'color': 'white', 'margin-left':'7px'}),
                    html.Br(),                        
                ], width=2, className="bg-primary text-black"),
                dbc.Col([
                        dcc.Graph(id='SPCDELTA_line'),
                        #html.Br(),
                        dcc.Graph(id='WBT_GUN'),
                        #html.Br(),
                        dcc.Graph(id='OFFSET')
                        ], width=5),
                dbc.Col([
                        dcc.Graph(id='W_DEPTH'),
                        #html.Br(),
                        dcc.Graph(id='TIDE'),
                        #html.Br(),
                        dcc.Graph(id='TYPE_bar')
                        ], width=5)
            ])
        ], label='Sequence-by-Sequence Header QC'),
        dbc.Tab([
            dbc.Row([
                dbc.Col([
                        dcc.Graph(figure=display_wbt3d())
                ], width = 6),
                dbc.Col([
                        dcc.Graph(figure=display_wbtgun3d())
                ], width = 6)    
            ])
        ], label='Global Header QC')
    ])
])     

## Callback for line information: 
@app.callback(Output('ffidrange', 'children'), 
              Output('sprange', 'children'),
              Output('sline', 'children'),
              Input('ACQSEQ_dropdown', 'value'))
def line_stats(ACQSEQ):
    if not ACQSEQ:
       raise PreventUpdate
    data = df1[df1['ACQSEQ'] == int(ACQSEQ)]
    ffidmin = data['FFID'].min() 
    ffidmax = data['FFID'].max()
    spmin = data['SHOT'].min() 
    spmax = data['SHOT'].max()
    sline = data['SLINE'].max()
    return f"FFID Range: {ffidmin} - {ffidmax}", f"SP Range : {spmin} - {spmax}", f"SLINE : {sline}"

## Callback for SPCDELTA line plot:
@app.callback(Output('SPCDELTA_line', 'figure'), Input('ACQSEQ_dropdown', 'value'))
def display_spc_delta(ACQSEQ):
    if not ACQSEQ:
       raise PreventUpdate
    # data = df1[[int(ACQSEQ), 'FFID', 'SPCDELTA']]
    data = df1[df1['ACQSEQ'] == int(ACQSEQ)]
    # Line plot for SPCDELTA
    chart = px.line(data,
                   x='FFID',
                   y='SPCDELTA',
                   range_x=['FFID_MIN','FFID_MAX'], range_y=[6000,11000], 
                   title=f'SPC_DELTA for Sequence: {ACQSEQ}'
                   )
    chart.add_hrect(y0=6000, y1=7000, line_width=0, fillcolor="red", opacity=0.2)
    chart.update_layout(plot_bgcolor='lavender')
    chart.update_traces(line_color='RebeccaPurple')
    chart.update_xaxes(linewidth = 2, linecolor ='black')
    chart.update_yaxes(linewidth = 2, linecolor = 'black')
    chart.update_layout(margin=dict(l=20, r=20, t=60, b=20))
    chart.update_layout(autosize=False, height=400, width =960)
    return chart

## Callback for WBT with GUN line plot:
@app.callback(Output('WBT_GUN', 'figure'), Input('ACQSEQ_dropdown', 'value'))
def display_wbt_gun(ACQSEQ):
    if not ACQSEQ:
       raise PreventUpdate
    data = df1[df1['ACQSEQ'] == int(ACQSEQ)]
    ymin = data['WBT'].min() - 50
    ymax = data['WBT'].max() + 50
    xmax = data['FFID'].max()
    xmin = data['FFID'].min()
    chart = px.scatter(data,
                   x='FFID',
                   y='WBT',
                   color="GUNFLAG", 
                   range_x=[xmin,xmax], 
                   range_y=[ymin,ymax], 
                   title=f'WBT for Each Gun - Sequence: {ACQSEQ}'
                   )
    #chart.add_hrect(y0=6000, y1=7000, line_width=0, fillcolor="red", opacity=0.2)
    chart.update_layout(plot_bgcolor='lavender')
    chart.update_xaxes(linewidth = 2, linecolor ='black')
    chart.update_yaxes(linewidth = 2, linecolor = 'black')
    chart.update_layout(margin=dict(l=20, r=20, t=40, b=20))
    chart.update_layout(autosize=False, height=400)
    return chart

## Callback for OFFSET line plot with GUNFLAG
@app.callback(Output('OFFSET', 'figure'), Input('ACQSEQ_dropdown', 'value'))
def display_offset(ACQSEQ):
    if not ACQSEQ:
       raise PreventUpdate
    data = df1[df1['ACQSEQ'] == int(ACQSEQ)]
    ymin = data['OFFSET'].min() - 10
    ymax = data['OFFSET'].max() + 10
    chart = px.line(data,
                   x='FFID',
                   y='OFFSET',
                   color="GUNFLAG",
                   color_discrete_sequence=['RebeccaPurple', 'MediumPurple'],
                   range_x=['FFID_MIN','FFID_MAX'], range_y=[ymin,ymax], 
                   title=f'OFFSET for each GUNFLAG - Sequence: {ACQSEQ}'
                   )
    #chart.add_hrect(y0=6000, y1=7000, line_width=0, fillcolor="red", opacity=0.2)
    chart.update_layout(plot_bgcolor='lavender')
    chart.update_xaxes(linewidth = 2, linecolor ='black')
    chart.update_yaxes(linewidth = 2, linecolor = 'black')
    chart.update_layout(margin=dict(l=20, r=20, t=40, b=20))
    chart.update_layout(autosize=False, height=400)
    return chart

## Callback for TIDE Scatter plot
@app.callback(Output('TIDE', 'figure'), Input('ACQSEQ_dropdown', 'value'))
def display_tide_msl(ACQSEQ):
    if not ACQSEQ:
       raise PreventUpdate
    data = df1[df1['ACQSEQ'] == int(ACQSEQ)]
    ymin = data['MSL_TIDE'].min() - 0.1
    ymax = data['MSL_TIDE'].max() + 0.1
    xmax = data['FFID'].max()
    xmin = data['FFID'].min()
    chart = px.scatter(data,
                   x='FFID',
                   y='MSL_TIDE',
                   color="SWEVEL", 
                   range_x=[xmin,xmax], range_y=[ymin,ymax], 
                   title=f'MSL TIDE with SWEVEL - Sequence: {ACQSEQ}'
                   )
    #chart.add_hrect(y0=6000, y1=7000, line_width=0, fillcolor="red", opacity=0.2)
    chart.update_layout(plot_bgcolor='lavender')
    chart.update_xaxes(linewidth = 2, linecolor ='black')
    chart.update_yaxes(linewidth = 2, linecolor = 'black')
    chart.update_layout(margin=dict(l=20, r=20, t=40, b=20))
    chart.update_layout(autosize=False, height=400)
    return chart

## Callback for the W_DEPTH line plot
@app.callback(Output('W_DEPTH', 'figure'), Input('ACQSEQ_dropdown', 'value'))
def display_wdepth(ACQSEQ):
    if not ACQSEQ:
       raise PreventUpdate
    data = df1[df1['ACQSEQ'] == int(ACQSEQ)]
    ymin = data['W_DEPTH'].min() - 50
    ymax = data['W_DEPTH'].max() + 50
    chart = px.line(data,
                   x='FFID',
                   y=['WDS', 'WDR', 'W_DEPTH'],
                   color_discrete_map={"WDS": "MediumPurple", "WDR": "gold", "W_DEPTH":"RebeccaPurple"}, 
                   range_x=['FFID_MIN','FFID_MAX'], range_y=[ymin,ymax], 
                   title=f'W_DEPTH / WDR / WDS - Sequence: {ACQSEQ}'
                   )
    #chart.add_hrect(y0=6000, y1=7000, line_width=0, fillcolor="red", opacity=0.2)
    chart.update_layout(plot_bgcolor='lavender')
    chart.update_xaxes(linewidth = 2, linecolor ='black')
    chart.update_yaxes(linewidth = 2, linecolor = 'black')
    chart.update_layout(margin=dict(l=20, r=20, t=60, b=20))
    chart.update_layout(autosize=False, height=400)
    return chart

## Callback for the TYPE bar chart
@app.callback(Output('TYPE_bar', 'figure'), Input('ACQSEQ_dropdown', 'value'))
def display_shot_type(ACQSEQ):
    if not ACQSEQ:
       raise PreventUpdate
    data = df1[df1['ACQSEQ'] == int(ACQSEQ)]
    ymin = 0
    ymax = 2
    chart = px.bar(data,
                   x='FFID',
                   y='TYPE',
                   color="GUNFLAG", 
                   range_x=['FFID_MIN','FFID_MAX'], range_y=[ymin,ymax],
                   color_continuous_scale=[(0, "Gold"), (1, "RebeccaPurple")], 
                   title=f'SHOT TYPE with GUNFLAG - Sequence: {ACQSEQ}'
                   )
    #chart.add_hrect(y0=6000, y1=7000, line_width=0, fillcolor="red", opacity=0.2)
    chart.update_layout(plot_bgcolor='lavender')
    chart.update_xaxes(linewidth = 2, linecolor ='black')
    chart.update_yaxes(linewidth = 2, linecolor = 'black')
    chart.update_layout(margin=dict(l=20, r=20, t=40, b=20))
    chart.update_layout(autosize=False, height=400)
    return chart


if __name__=='__main__':
    app.run_server(mode='inline', port = 8213)
   

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Dropdown(id='ACQSEQ_dropdown', options=[{'label': ACQSEQ, 'value': str(ACQSEQ)}
                                     for ACQSEQ in df1['ACQSEQ'].drop_duplicates().sort_values()]),
    html.Br(),
    html.P(id='sline', style={'color': 'white'} ),
    html.P(id='ffidrange', style={'color': 'white'} ),
    html.P(id='sprange', style={'color': 'white'}),
    html.Br(),

    #dbc.Select(id='LINE_STATS')
])

@app.callback(Output('ffidrange', 'children'), 
              Output('sprange', 'children'),
              Output('sline', 'children'),
              Input('ACQSEQ_dropdown', 'value'))
def line_stats(ACQSEQ):
    if not ACQSEQ:
       raise PreventUpdate
    data = df1[df1['ACQSEQ'] == int(ACQSEQ)]
    ffidmin = data['FFID'].min() 
    ffidmax = data['FFID'].max()
    spmin = data['SHOT'].min() 
    spmax = data['SHOT'].max()
    sline = data['SLINE'].max()
    return f"FFID Range: {ffidmin} - {ffidmax}", f"SP Range : {spmin} - {spmax}", f"SLINE : {sline}"

if __name__=='__main__':
    app.run_server(mode='inline', port = 8124)

#### The Graphical functions tested... 

In [ ]:
app = JupyterDash(__name__)


# @app.callback(Output('WBT_mesh', 'figure'), Input(' ', 'value'))
def display_wbt3d():
    data1 = df1
    minx = data1['SHT_X'].min()
    miny = data1['SHT_Y'].min()
    maxx = data1['SHT_X'].max()
    maxy = data1['SHT_Y'].max()
    minz = data1['WBT'].min()
    maxz = data1['WBT'].max()
    z=pd.Series(data1['WBT'])
    x=pd.Series(data1['SHT_X'])
    y=pd.Series(data1['SHT_Y'])
    # 
    # print(x)
    chart1 = go.Figure(data = [go.Mesh3d(
                   z=z,
                   x=x,
                   y=y, 
                   opacity=0.25,
                   color='purple'
                   )])
    chart1.update_layout(title='WBT: Shot X-Y', autosize=False,
                  width=1000, height=1000,
                  scene = dict(
                        xaxis = dict(nticks=8, range=[minx,maxx],),
                        yaxis = dict(nticks=8, range=[miny,maxy],),
                        zaxis = dict(nticks=8, range=[maxz,minz],),
                        camera_eye = {"x": 1.5, "y": -1.5, "z": 0.5},
                        ),
                  margin=dict(l=35, r=20, b=35, t=40))
    return chart1

app.layout = html.Div([
        dcc.Graph(
            figure=display_wbt3d()
            #id='WBT_mesh',
            #className="figure"
        )
])

if __name__=='__main__':
    app.run_server(mode='inline', port = 8113)

In [79]:
data1['SHT_X'].min()



414460.40625

In [74]:
print(display_spc_delta())


0      428532.12500
1      428529.56250
2      428499.84375
3      428498.15625
4      428468.90625
           ...     
873    428319.37500
874    428354.75000
875    428351.68750
876    428386.59375
877    428384.62500
Name: SHT_X, Length: 55889, dtype: float64


In [ ]:
x = df1['SHT_X']
y = df1['SHT_Y']
z = df1['WBT']
color = df1['GUNFLAG']
maxx = max(x)
minx = 419000
maxy = max(y)
miny = 9184000
maxz = max(z)
minz = 2000
fig = go.Figure(data=[go.Mesh3d(z=z, x=x, y=y, opacity=0.25, color='purple')])

fig.update_layout(title='WBT: Shot X-Y', autosize=False,
                  width=1000, height=1000,
                  scene = dict(
                        xaxis = dict(nticks=8, range=[minx,maxx],),
                        yaxis = dict(nticks=8, range=[miny,maxy],),
                        zaxis = dict(nticks=8, range=[maxz,minz],),
                        camera_eye = {"x": 1.5, "y": -1.5, "z": 0.5},
                        ),
                  margin=dict(l=35, r=20, b=35, t=40))
fig.write_html("2022057_WBT_QC.html")
fig.show()

In [6]:
max(y)

9198468.0

In [ ]:
app = JupyterDash(__name__)


# @app.callback(Output('WBT_mesh', 'figure'), Input(' ', 'value'))
def display_wbtgun3d():
    data1 = df1
    minx = data1['SHT_X'].min()
    miny = data1['SHT_Y'].min()
    maxx = data1['SHT_X'].max()
    maxy = data1['SHT_Y'].max()
    minz = data1['WBT'].min()
    maxz = data1['WBT'].max()
    z=pd.Series(data1['WBT'])
    x=pd.Series(data1['SHT_X'])
    y=pd.Series(data1['SHT_Y'])
    color = data1['GUNFLAG']
    # 
    # print(x)
    chart1 = px.line_3d(
                   z=z,
                   x=x,
                   y=y, 
                   color=color
                   )
    chart1.update_layout(title='WBT with GUNFLAG: Shot X-Y (Click legend to swap guns)', autosize=False,
                  width=1000, height=1000,
                  scene = dict(
                        xaxis = dict(nticks=8, range=[minx,maxx],),
                        yaxis = dict(nticks=8, range=[miny,maxy],),
                        zaxis = dict(nticks=8, range=[maxz,minz],),
                        camera_eye = {"x": 1.5, "y": -1.5, "z": 0.5},
                        ),
                  margin=dict(l=35, r=20, b=35, t=40))
    return chart1

app.layout = html.Div([
        dcc.Graph(
            figure=display_wbtgun3d()
            #id='WBT_mesh',
            #className="figure"
        )
])

if __name__=='__main__':
    app.run_server(mode='inline', port = 8109)

In [ ]:
x = df1['SHT_X']
y = df1['SHT_Y']
z = df1['WBT']
color = df1['GUNFLAG']
maxx = max(x)
# minx = min(x)
minx = 419000
maxy = max(y)
# miny = 9184000
# miny = min(y)
maxz = max(z)
## minz = min(z)
minz = 2000
fig = px.line_3d(z=z, x=x, y=y, color=color)

fig.update_layout(title='WBT: Shot X-Y (Click legend to swap guns)', autosize=False,
                  width=1000, height=1000,
                  scene = dict(
                        xaxis = dict(nticks=8, range=[minx,maxx],),
                        yaxis = dict(nticks=8, range=[miny,maxy],),
                        zaxis = dict(nticks=8, range=[maxz,minz],),
                        camera_eye = {"x": 1.5, "y": -1.5, "z": 0.5},
                        ),
                  margin=dict(l=35, r=20, b=35, t=40))
fig.update_layout(legend=dict(
    title='GUNFLAG'
    #yanchor="top",
    #y=0.99,
    #xanchor="left",
    #x=0.01
))
fig.write_html("2022057_WBT_GUNFLAG_ALL_QC.html")
fig.show()

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Dropdown(id='ACQSEQ_dropdown', options=[{'label': ACQSEQ, 'value': str(ACQSEQ)}
                                     for ACQSEQ in df1['ACQSEQ'].drop_duplicates().sort_values()]),
    dcc.Graph(id='WBT_GUN')
])

@app.callback(Output('WBT_GUN', 'figure'), Input('ACQSEQ_dropdown', 'value'))
def display_wbt_gun(ACQSEQ):
    if not ACQSEQ:
       raise PreventUpdate
    data = df1[df1['ACQSEQ'] == int(ACQSEQ)]
    
    chart = px.scatter(data,
                   x='FFID',
                   y='WBT',
                   color="GUNFLAG", 
                   range_x=['FFID_MIN','FFID_MAX'], range_y=[2000,2500], 
                   title=f'WBT for Each Gun - Sequence: {ACQSEQ}'
                   )
    chart.update_layout(margin=dict(l=20, r=20, t=40, b=20))
    #chart.add_hrect(y0=6000, y1=7000, line_width=0, fillcolor="red", opacity=0.2)
    return chart
if __name__=='__main__':
    app.run_server(mode='inline', port = 8104)

In [ ]:
fig = px.scatter(df, x="FFID", y="WBT" , color="GUNFLAG", title="WBT for Each Gun", 
                    animation_frame="ACQSEQ", 
                    range_x=[10,1000], range_y=[2000,2500] )
fig.write_html("2022057_WBT_GUNFLAG_QC.html")
fig.show()

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Dropdown(id='ACQSEQ_dropdown', options=[{'label': ACQSEQ, 'value': str(ACQSEQ)}
                                     for ACQSEQ in df1['ACQSEQ'].drop_duplicates().sort_values()]),
    dcc.Graph(id='W_DEPTH')
])

@app.callback(Output('W_DEPTH', 'figure'), Input('ACQSEQ_dropdown', 'value'))
def display_wbt_gun(ACQSEQ):
    if not ACQSEQ:
       raise PreventUpdate
    data = df1[df1['ACQSEQ'] == int(ACQSEQ)]
    ymin = data['W_DEPTH'].min() - 50
    ymax = data['W_DEPTH'].max() + 50
    chart = px.line(data,
                   x='FFID',
                   y=['WDS', 'WDR', 'W_DEPTH'],
                   #color="GUNFLAG", 
                   range_x=['FFID_MIN','FFID_MAX'], range_y=[ymin,ymax], 
                   title=f'W_DEPTH / WDR / WDS - Sequence: {ACQSEQ}'
                   )
    #chart.add_hrect(y0=6000, y1=7000, line_width=0, fillcolor="red", opacity=0.2)
    return chart
if __name__=='__main__':
    app.run_server(mode='inline', port = 8115)

In [ ]:
# import plotly.graph_objects as go

fig = px.line(df1, x='FFID', y=['WDS', 'WDR', 'W_DEPTH'], title="Water Bottom Depth",
                    animation_frame="ACQSEQ", 
                    range_x=[10,1000], range_y=[1500,1900] )
fig.write_html("2022057_WBS_WDR_WDEPTH_QC.html")

fig.show()

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Dropdown(id='ACQSEQ_dropdown', options=[{'label': ACQSEQ, 'value': str(ACQSEQ)}
                                     for ACQSEQ in df1['ACQSEQ'].drop_duplicates().sort_values()]),
    dcc.Graph(id='OFFSET')
])

@app.callback(Output('OFFSET', 'figure'), Input('ACQSEQ_dropdown', 'value'))
def display_wbt_gun(ACQSEQ):
    if not ACQSEQ:
       raise PreventUpdate
    data = df1[df1['ACQSEQ'] == int(ACQSEQ)]
    ymin = data['OFFSET'].min() - 10
    ymax = data['OFFSET'].max() + 10
    chart = px.line(data,
                   x='FFID',
                   y='OFFSET',
                   color="GUNFLAG", 
                   range_x=['FFID_MIN','FFID_MAX'], range_y=[ymin,ymax], 
                   title=f'W_DEPTH / WDR / WDS - Sequence: {ACQSEQ}'
                   )
    #chart.add_hrect(y0=6000, y1=7000, line_width=0, fillcolor="red", opacity=0.2)
    return chart
if __name__=='__main__':
    app.run_server(mode='inline', port = 8116)

In [ ]:
fig = px.scatter(df, x="FFID", y="OFFSET" , color="GUNFLAG", title="Offset for Each Gun",
                        animation_frame="ACQSEQ", 
                        range_x=[10,1000], range_y=[180,290])
fig.write_html("2022057_OFFSET_GUNFLAG12_QC.html")
fig.show()

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Dropdown(id='ACQSEQ_dropdown', options=[{'label': ACQSEQ, 'value': str(ACQSEQ)}
                                     for ACQSEQ in df1['ACQSEQ'].drop_duplicates().sort_values()]),
    dcc.Graph(id='TIDE')
])

@app.callback(Output('TIDE', 'figure'), Input('ACQSEQ_dropdown', 'value'))
def display_wbt_gun(ACQSEQ):
    if not ACQSEQ:
       raise PreventUpdate
    data = df1[df1['ACQSEQ'] == int(ACQSEQ)]
    ymin = data['MSL_TIDE'].min() - 0.1
    ymax = data['MSL_TIDE'].max() + 0.1
    chart = px.scatter(data,
                   x='FFID',
                   y='MSL_TIDE',
                   color="SWEVEL", 
                   range_x=['FFID_MIN','FFID_MAX'], range_y=[ymin,ymax], 
                   title=f'MSL TIDE with SWEVEL - Sequence: {ACQSEQ}'
                   )
    #chart.add_hrect(y0=6000, y1=7000, line_width=0, fillcolor="red", opacity=0.2)
    return chart
if __name__=='__main__':
    app.run_server(mode='inline', port = 8117)

In [ ]:
fig = px.scatter(df, x="FFID", y="MSL_TIDE" , color="SWEVEL", title="Tide Data with Velocity at Cable Depth",
                        animation_frame="ACQSEQ", 
                        range_x=[10,1000], range_y=[-0.75,0.75])
fig.write_html("2022057_MSL_TIDE_SWEVEL_QC.html")
fig.show()

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Dropdown(id='ACQSEQ_dropdown', options=[{'label': ACQSEQ, 'value': str(ACQSEQ)}
                                     for ACQSEQ in df1['ACQSEQ'].drop_duplicates().sort_values()]),
    dcc.Graph(id='TYPE_bar')
])

@app.callback(Output('TYPE_bar', 'figure'), Input('ACQSEQ_dropdown', 'value'))
def display_wbt_gun(ACQSEQ):
    if not ACQSEQ:
       raise PreventUpdate
    data = df1[df1['ACQSEQ'] == int(ACQSEQ)]
    ymin = 0
    ymax = 2
    chart = px.bar(data,
                   x='FFID',
                   y='TYPE',
                   color="GUNFLAG", 
                   range_x=['FFID_MIN','FFID_MAX'], range_y=[ymin,ymax],
                   color_continuous_scale=[(0, "red"), (1, "blue")], 
                   title=f'SHOT TYPE with GUNFLAG - Sequence: {ACQSEQ}'
                   )
    #chart.add_hrect(y0=6000, y1=7000, line_width=0, fillcolor="red", opacity=0.2)
    return chart
if __name__=='__main__':
    app.run_server(mode='inline', port = 8118)

In [ ]:
fig = px.bar(df, x="FFID", y="TYPE" , color="GUNFLAG", 
             animation_frame="ACQSEQ",
             range_x=[10,1000], range_y=[0,2],
                          color_continuous_scale='RdBu', template='ggplot2')
fig.show()